## 章節 3：估計 (Estimation)

### 3.5 信賴區間（Credible Intervals）

一但你計算了後驗分佈，通常會用以下方式做總結：

1. 點估計：通常是計算後驗分佈的期望值、中位數或是那個結果有最大的似然性（Maximum likelihood）
1. 區間估計

對於區間，我們會計算出兩個數值找出區間，對於某一個未知數值有 90% 的機率（或是其他的機率）落在此區間。這兩個數值就代表一個信賴區間（Credible Interval）

一個簡單得方式計算信賴區間是加總後驗分佈的機率，並且紀錄對應加總機率為5%跟95%。換句話說就是第 5 百分位（percentiles）跟第 95 百分位。

thinkbayes.py 提供了方法計算百分位。

<pre>
 def Percentile(pmf, percentage):
    p = percentage / 100.0
    total = 0
    for val, prob in pmf.Items():
        total += prob
        if total >= p:
            return val
</pre>

請參考下方程式碼的 expected_values 方法內，有利用 Percentile 方法的範例。

In [6]:
from thinkbayes import Suite, Percentile


class Train(Suite): # 將均勻分佈作為先驗分佈
    
    def Likelihood(self, data, hypo):
        if data > hypo:
            return 0.0

        return 1.0/hypo


class Train_PowerLaw(Train):
    
    def __init__(self, hypos, alpha=1.0):
        super().__init__()
        for hypo in hypos: # 將冪律分佈作為先驗分佈
            self.Set(hypo, hypo**-alpha)
            
        self.Normalize()
    

def expected_values(hypos, data):
    suite = Train_PowerLaw(hypos)
    for datum in data:
        suite.Update(datum)
        
    print("假設範圍", hypos[0], "~", hypos[-1], "在看到編號", data,"後的期望值:", suite.Mean())
    
    interval = Percentile(suite, 5), Percentile(suite, 95) # 增加信賴區間90%的估計
    print("信賴區間", interval)
    print()
    
    
    
data = [60, 30, 90]
expected_values(range(1, 1001), data)
expected_values(range(1, 501), data)
expected_values(range(1, 2001), data)
expected_values(range(1, 100001), data)

假設範圍 1 ~ 1000 在看到編號 [60, 30, 90] 後的期望值: 133.2752313750312
信賴區間 (91, 242)

假設範圍 1 ~ 500 在看到編號 [60, 30, 90] 後的期望值: 130.70846986256004
信賴區間 (91, 235)

假設範圍 1 ~ 2000 在看到編號 [60, 30, 90] 後的期望值: 133.99746308073065
信賴區間 (91, 243)

假設範圍 1 ~ 100000 在看到編號 [60, 30, 90] 後的期望值: 134.25408214850083
信賴區間 (91, 243)



看到之前的火車頭例子，觀察上方結果得知。用冪律分佈作為先驗分佈，90%的信賴區間是 (91, 243)。雖然有個90%的信賴區間，但這個範圍的寬度還是讓我們仍然不確定有多少火車頭。